### Reading into the DataFrame

In [ ]:
df_2017 = pd.read_csv('./Reddit_data/Combined_Yearly_Data/stage2_fulltext2017.csv')
df_2018 = pd.read_csv('./Reddit_data/Combined_Yearly_Data/stage2_fulltext2018.csv')
df_2019 = pd.read_csv('./Reddit_data/Combined_Yearly_Data/stage2_fulltext2019.csv')
df_2020 = pd.read_csv('./Reddit_data/Combined_Yearly_Data/stage2_fulltext2020.csv')

In [1]:
df = pd.read_csv('./Reddit_data/Combined_Yearly_Data/stage4_sent2020.csv')

<IPython.core.display.Javascript object>

### Checking for Null Values

In [5]:
removenull(df)

In [4]:
def removenull(df):
    if df['title'].isnull().sum() != 0:
        df.dropna(subset=['title'], inplace = True)

### Concatenating Titles and Self Text

In [ ]:
def textcombine(df):
    df['title_and_text'] = ''
    for i in range(0, len(df['title'])):
        if pd.isnull(df['selftext'][i]) == True:
            df['title_and_text'][i] = df['title'][i]
        elif df['selftext'][i] == '[removed]':
            df['title_and_text'][i] = df['title'][i]
        else:
            df['title_and_text'][i] = df['title'][i] + ' ' + df['selftext'][i]

### Text Normalization

In [ ]:
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [ ]:
def standardize_text(df, column):
    for i in range(len(df)):

        review_text = str(BeautifulSoup(df[column][i]).get_text())

        letters_only = re.sub("[^a-zA-Z0-9]", " ", review_text)


        words = letters_only.lower().split()

        stops = set(stopwords.words('english'))


        meaningful_words = [w for w in words if w not in stops]

        df[column][i] = " ".join(meaningful_words)

###  Sentiment Analyzer (Vader, Textblob, Flair) (Spacy, StanfordNLP)

In [10]:
from nltk.classify import NaiveBayesClassifier
from nltk.corpus import subjectivity
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment.util import *
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
from textblob.sentiments import NaiveBayesAnalyzer
import flair
from flair.data import Sentence
from flair.models import SequenceTagger

In [ ]:
sid = SentimentIntensityAnalyzer()

In [ ]:
def vader_score(df, column):
    
    df["vader_neu_score"] = ""
    df["vader_pos_score"] = ""
    df["vader_neg_score"] = ""
    df["vader_compound"] = ""
    
    for i in range(0, len(df)):
        sid = SentimentIntensityAnalyzer()
        sent_dict = sid.polarity_scores(df[column][i])
        df["vader_neg_score"][i] = sent_dict['neg']
        df["vader_neu_score"][i] = sent_dict['neu']
        df["vader_pos_score"][i] = sent_dict['pos']
        df["vader_compound"][i] = sent_dict['compound']

In [11]:
def blob_score(df, column):
    
    df['blob_polarity'] = ''
    df['blob_subjectivity'] = ''
    
    for i in range(len(df)):
        blob = TextBlob(df[column][i])
        df['blob_polarity'][i] = blob.sentiment[0]
        df['blob_subjectivity'][i] = blob.sentiment[1]

In [12]:
blob_score(df, 'title_and_text')

C:\Users\bhuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\bhuan\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [13]:
df

,subreddit,created_utc,title,selftext,score,Date,title_and_text,vader_neu_score,vader_pos_score,vader_neg_score,vader_compound,textblob_score,blob_polarity,blob_subjectivity,flair_polarity,flair_score
0,Bitcoin,1577855217,(20) BTC Pay Server: Pairing your Exodus Walle...,NaN,1,2020-01-01,(20) BTC Pay Server: Pairing your Exodus Walle...,0.896,0.000,0.104,-0.1027,NaN,0,0,NEGATIVE,0.997896
1,Bitcoin,1577855404,Happy New Year Crypto family,NaN,1,2020-01-01,Happy New Year Crypto family,0.519,0.481,0.000,0.5719,NaN,0.468182,0.727273,POSITIVE,0.999864
2,Bitcoin,1577857479,Whaddaya think? Is this how our financial syst...,NaN,1,2020-01-01,Whaddaya think? Is this how our financial syst...,0.796,0.204,0.000,0.6544,NaN,0.323485,0.484848,NEGATIVE,0.993958
3,Bitcoin,1577860038,Happy New Year to All!,I hope this year will be more Happy and Healt...,1,2020-01-01,Happy New Year to All! I hope this year will ...,0.470,0.530,0.000,0.9268,NaN,0.554091,0.690909,POSITIVE,0.999989
4,Bitcoin,1577860986,2k gifted in bitcoin? Scam?,&amp;#x200B;\n\n[not sure if this is legit the...,1,2020-01-01,2k gifted in bitcoin? Scam? &amp;#x200B;\n\n[n...,0.788,0.132,0.081,-0.1506,NaN,0.125,0.944444,NEGATIVE,0.999743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12251,Bitcoin,1583679513,How North Korean hackers exploit Bitcoin’s blo...,NaN,1,2020-03-08,How North Korean hackers exploit Bitcoin’s blo...,0.877,0.000,0.123,-0.1027,NaN,0,0,NaN,NaN
12252,Bitcoin,1583680203,Coinbase CEO Makes Serious Warning,NaN,1,2020-03-08,Coinbase CEO Makes Serious Warning,0.448,0.000,0.552,-0.4019,NaN,-0.333333,0.666667,NaN,NaN
12253,Bitcoin,1583680221,New CRYPTOCURRENCY - EXITSCAM,NaN,1,2020-03-08,New CRYPTOCURRENCY - EXITSCAM,1.000,0.000,0.000,0.0000,NaN,0.136364,0.454545,NaN,NaN
12254,Bitcoin,1583681553,People Are Getting Bored Already Of This Price...,https://u.today/bitcoin-btc-advice-from-top-tr...,1,2020-03-08,People Are Getting Bored Already Of This Price...,0.826,0.000,0.174,-0.2732,NaN,-0.5,1,NaN,NaN


In [ ]:
def flair_score(df, column):
    
    flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
    df['flair_polarity'] = ""
    df['flair_score'] = ""
    
    for i in range(0, len(df)):
        s = flair.data.Sentence(df[column][i])
        flair_sentiment.predict(s)
        total_sentiment = s.labels
        total_sentiment
        df['flair_polarity'][i] = str(total_sentiment[0]).split(' (')[0]
        df['flair_score'][i] = (str(total_sentiment[0]).split(' (')[1].split(')')[0])

We can see that polarity is 0.8, which means that the statement is positive and 0.75 subjectivity refers that mostly it is a public opinion and not a factual information.

### Making Mean Sentiment Scores

In [ ]:
df['neu_score'] = df['neu_score'].astype(str).astype(float)
df['pos_score'] = df['pos_score'].astype(str).astype(float)
df['neg_score'] = df['neg_score'].astype(str).astype(float)
df['compound'] = df['compound'].astype(str).astype(float)

In [ ]:
mean_df = df.groupby('Date').agg('mean')

### Counting Posts Per day

In [ ]:
df['Sum_posts'] = 1

In [ ]:
sum_df = df.groupby('Date').agg('sum')

In [ ]:
sum_df.drop(columns = sum_df.columns[:-1], inplace = True)

In [ ]:
df2 = pd.merge(mean_df, sum_df, left_index = True, right_index = True)

In [ ]:
mini_sum['zip_code'] = mini_sum.index

In [ ]:
df2.to_csv('summation_final17.csv', index = True)

### Visuals

In [ ]:
plt.plot(mean_df['compound'], label = 'Compound Score')
plt.axhline(y=0.05, color='r', linestyle='-')
plt.axhline(y=-0.05, color='b', linestyle='-')
plt.legend()